In [ ]:
!pip install ctgan pandas scikit-learn

In [ ]:
# ✅ CTGAN theo từng Fault_Type + Gán thời gian từ lỗi thật (có inverse_transform)

import pandas as pd
from ctgan import CTGAN
from sklearn.preprocessing import StandardScaler
from google.colab import files
import numpy as np

# ⬇️ Tải file từ máy

df = pd.read_csv("/content/processed_iot_dataset.csv")

sensor_cols = ['Temperature', 'Vibration', 'Pressure', 'Voltage', 'Current',
               'FFT_Feature1', 'FFT_Feature2', 'Anomaly_Score']

scaler = StandardScaler()
generated = []

for fault_type in [1, 2, 3]:
    sub = df[(df['Fault_Status'] == 1) & (df['Fault_Type'] == fault_type)].copy()
    if len(sub) < 100:
        continue

    data_scaled = scaler.fit_transform(sub[sensor_cols])
    df_scaled = pd.DataFrame(data_scaled, columns=sensor_cols)
    df_scaled['Fault_Type'] = fault_type

    print(f"⏳ Đang huấn luyện CTGAN cho lỗi loại {fault_type} ({len(df_scaled)} mẫu)...")
    ctgan = CTGAN(epochs=300)
    ctgan.fit(df_scaled, discrete_columns=['Fault_Type'])

    synth = ctgan.sample(len(df_scaled))
    synth['Fault_Status'] = 1
    synth['Fault_Type'] = fault_type

    # 🔁 Hoàn tác chuẩn hóa → loại bỏ giá trị âm
    synth[sensor_cols] = scaler.inverse_transform(synth[sensor_cols])

    generated.append(synth)

# Gộp lại
df_synth_all = pd.concat(generated, ignore_index=True)

# ⏱ Gán thời gian từ dữ liệu lỗi gốc
df_real_times = df[df['Fault_Status'] == 1][['Year', 'Month', 'Day', 'Hour', 'Minute']]
random_times = df_real_times.sample(n=len(df_synth_all), replace=True, random_state=42).reset_index(drop=True)
df_synth_all = pd.concat([random_times, df_synth_all.reset_index(drop=True)], axis=1)

# 🆔 Gán Sensor_ID
df_synth_all['Sensor_ID'] = np.random.randint(100, 200, size=len(df_synth_all))

# 💾 Lưu kết quả
df_synth_all.to_csv("ctgan_generated_faults_by_type_with_time.csv", index=False)
files.download("ctgan_generated_faults_by_type_with_time.csv")


⏳ Đang huấn luyện CTGAN cho lỗi loại 1 (5034 mẫu)...
⏳ Đang huấn luyện CTGAN cho lỗi loại 2 (4950 mẫu)...
⏳ Đang huấn luyện CTGAN cho lỗi loại 3 (5028 mẫu)...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>